In [11]:
import pandas
import time
import numpy as np
import preprocessors

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import IncrementalPCA

from estimators import LSHNearestNeighbors

In [12]:
path = "/media/alexander/b32bf4b4-8724-4107-9d19-abf6615c2f60/alexander/HELP_FILE/query.yaHotelId.showInTop.sure.final.tsv"

In [13]:
df = pandas.read_csv(path, sep="\t")
print("Изначальная размерность данных:", df.shape,";", "Количество отелей:", len(df["yaHotelId"].unique()))

Изначальная размерность данных: (588232, 4) ; Количество отелей: 95884


In [14]:
sure_df = df[df["sure"]]
filtered_values = [value[0] for value in sure_df["yaHotelId"].value_counts().iteritems() if value[1] >= 5]
filtered_df = sure_df[sure_df["yaHotelId"].isin(filtered_values)]
print("Получившаяся размерность данных:", filtered_df.shape, ";", "Количество отелей:", len(filtered_df["yaHotelId"].unique()))

Получившаяся размерность данных: (92427, 4) ; Количество отелей: 7696


Разделяем выборку на тренировочную и тестовую части

In [15]:
df_train, df_test = train_test_split(filtered_df, test_size=0.1)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

Преобразуем тренировочное множество:

Tf-idf преобразование:

In [16]:
prep = preprocessors.Preprocessor([preprocessors.mystem_preprocess])
vectorizer = TfidfVectorizer(preprocessor=prep.preprocess)
y_train = np.array(df_train["yaHotelId"])
X_train = vectorizer.fit_transform(df_train["query"])
X_train.shape

(83184, 15542)

In [18]:
y_test = np.array(df_test["yaHotelId"])
X_test = vectorizer.transform(df_test["query"])

Обучение:

In [19]:
clf = LSHNearestNeighbors(n_estimators=10, n_candidates=100, n_neighbors=9, mode="parzen_window")
clf.fit(X_train, y_train)

LSHNearestNeighbors(answers='one', mode='parzen_window', n_candidates=100,
          n_estimators=None, n_neighbors=9)

Предсказание:

In [20]:
y_pred = clf.predict(X_test)
print("accuracy:", accuracy_score(y_test, y_pred))

accuracy: 0.822784810127


Статистика по предсказанию:

In [21]:
clf.get_quantile(p=0.9)

0    0.107182
dtype: float64

Статистика по обработке:

In [22]:
prep.get_quantile(p=0.9)

0    0.001072
dtype: float64